In [1]:
import pandas as pd
import requests
from rich.progress import track
import ast
import numpy as np



In [2]:
df = pd.read_csv('swarfarm_spells.csv')

In [3]:
df

,name,description,slot,cooltime,hits,passive,aoe,multiplier_formula,multiplier_formula_raw,scales_with,...,upgrade_4_effect,upgrade_4_amount,upgrade_5_effect,upgrade_5_amount,upgrade_6_effect,upgrade_6_amount,upgrade_7_effect,upgrade_7_amount,upgrade_8_effect,upgrade_8_amount
0,Forbidden Ingredient (Passive),Increases the Skill Level of the monster by 1 ...,1,0.0,0,True,False,NaN,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,Special Ingredient (Passive),"The XP granted when used as Power-up material,...",1,0.0,0,True,False,NaN,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Best Ingredient (Passive),The XP granted will additionally increase by 5...,2,0.0,0,True,False,NaN,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",['ATK'],...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
4,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",['ATK'],...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,Pulverize,Strikes the enemy with a hammer and prevents t...,1,0.0,1,False,False,2.4*{ATK} + 2.8*{DEF},"[[""ATK"", ""*"", 2.4], [""+""], [""DEF"", ""*"", 2.8]]","['ATK', 'DEF']",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3796,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",['DEF'],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3797,Powerful Smash,"Slams the enemy with a hammer, decreasing the ...",2,3.0,1,False,False,3.7*{ATK} + 5.0*{DEF},"[[""ATK"", ""*"", 3.7], [""+""], [""DEF"", ""*"", 5.0]]","['ATK', 'DEF']",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3798,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",['DEF'],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [4]:
liste = df['slot'].unique().tolist()
liste.append('*')
liste

[1, 2, 3, 4, -1, '*']

In [5]:
from types import NoneType


def extraire_variables_imbriquees(df, colonne, rename=None):
    
    df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].items()]
    
    df = pd.concat([df.drop([colonne], axis=1), df[colonne].apply(pd.Series)], axis=1)


    return df

def changement_effect(x, number):
    if x != 0 and x!= -1 and x != "-1":
        effect = {key + number: x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def changement_lead(x):
    if x != 0 and x!= -1 and x != "-1" and not isinstance(x, NoneType):
        effect = {key + '_leader': x[key] for key in x} # modification key
    else:
        effect = 0
    return effect

def swarfarm_monstres():
        # database swarfarm
    url = "https://swarfarm.com/api/v2/monsters/?page=1"
    r = requests.get(url=url)
    data = r.json()
    df = pd.DataFrame(data)

    for i in track(range(2,25), description="Chargement de la Database Swarfarm..."):
        url = f"https://swarfarm.com/api/v2/monsters/?page={i}"
        r = requests.get(url=url)
        data = r.json()
        df2 = pd.DataFrame(data)
        df = pd.concat([df, df2])
        
    
    # on extrait les variables du dict dans la colonne ['results'] et on supprime ce qui m'intéresse pas
    df = extraire_variables_imbriquees(df, 'results')
    df.drop(['next', 'previous'], axis=1, inplace=True)
    # On garde ce qui nous intéresse
    df_mobs_swarfarm = df[['id', 'url', 'com2us_id', 'family_id', 'name', 'element', 'archetype', 'skills', 'leader_skill', 'image_filename', 'natural_stars', 'awaken_level']]
    
    df_mobs_swarfarm['leader_skill'] = df_mobs_swarfarm['leader_skill'].apply(changement_lead)
    
    df_mobs_swarfarm = extraire_variables_imbriquees(df_mobs_swarfarm, 'leader_skill')
    
    df_mobs_swarfarm.drop([0], axis=1, inplace=True)
    
    df_mobs_swarfarm.fillna({'url_leader' : 'Aucun',
                             'attribute_leader' : 'Aucun',
                             'amount_leader' : 'Aucun',
                             'area_leader' : 'Aucun',
                             'id_leader' : 'Aucun',
                             'element_leader' : 'Aucun'}, inplace=True)
    
    # on rajoute le type des monstres dans la colonne 'name' pour les monstres non-awaken
    
    df_mobs_swarfarm.reset_index(drop=True, inplace=True)
    df_mobs_swarfarm.loc[df_mobs_swarfarm['awaken_level'] == 0, 'name'] = df_mobs_swarfarm['name'] + ' ' + df_mobs_swarfarm['element']

    
    df_mobs_swarfarm.to_excel('swarfarm_monstres.xlsx', index=False)
    df_mobs_swarfarm.to_csv('swarfarm_monstres.csv', index=False)
    
    return df_mobs_swarfarm
    

In [6]:
df_mob = swarfarm_monstres()

Output()

C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1180144450.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mobs_swarfarm['leader_skill'] = df_mobs_swarfarm['leader_skill'].apply(changement_lead)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1180144450.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[colonne] = [ast.literal_eval(str(item)) for index, item in df[colonne].items()]


In [7]:
def spells_monstres():
        # database swarfarm
    url = "https://swarfarm.com/api/v2/skills/?page=1"
    r = requests.get(url=url)
    data = r.json()
    df = pd.DataFrame(data)

    for i in track(range(2,39), description="Chargement de la Database Spells Swarfarm..."):
        url = f"https://swarfarm.com/api/v2/skills/?page={i}"
        r = requests.get(url=url)
        data = r.json()
        df2 = pd.DataFrame(data)
        df = pd.concat([df, df2])
    return df    


In [8]:
df = spells_monstres()

Output()

In [9]:
 # on extrait les variables du dict dans la colonne ['results'] et on supprime ce qui m'intéresse pas
df = extraire_variables_imbriquees(df, 'results')
df.drop(['next', 'previous'], axis=1, inplace=True)
# On garde ce qui nous intéresse
    
df_spells = df[['id', 'description', 'slot', 'cooltime', 'hits', 'aoe', 'passive', 'effects', 'used_on', 'upgrades', 'multiplier_formula', 'scales_with']]
    
# 0 pour les valeurs nulles    
df_spells.fillna(0, inplace=True)
    
# on extrait les variables dans "effects"
df_spells = extraire_variables_imbriquees(df, 'effects')
    
# on set sur l'id du spell
df_spells.set_index('id', inplace=True)
    
# on remplace les valeurs nulles du effect par 0
df_spells.fillna(0, inplace=True)
    
# on rename les colonnes des effects
df_spells.rename(columns={0 : 'Effet1', 1 : 'Effet2', 2: 'Effet3', 3: 'Effet4', 4: 'Effet5'}, inplace=True)
    

    

            
# on modifie les colonnes effects pour plus de lisibilité
df_spells['Effet1'] = df_spells['Effet1'].apply(changement_effect, number="_1")
df_spells['Effet2'] = df_spells['Effet2'].apply(changement_effect, number="_2")
df_spells['Effet3'] = df_spells['Effet3'].apply(changement_effect, number="_3")
df_spells['Effet4'] = df_spells['Effet4'].apply(changement_effect, number="_4")
df_spells['Effet5'] = df_spells['Effet5'].apply(changement_effect, number="_5")

    
    
# on peut désormais extraire les variables des effects
for effect in ['Effet1', 'Effet2', 'Effet3', 'Effet4', 'Effet5']:
    df_spells = extraire_variables_imbriquees(df_spells, effect)
        
   
df_spells.drop([0], axis=1, inplace=True) 
       
    # si vide, pas d'effet, donc false
for num_effet in range(1,6):
    num_effet = str(num_effet)
    col_list = ['effect_' + num_effet, 'aoe_' + num_effet, 'single_target_' + num_effet,
            'self_effect_' + num_effet, 'chance_' + num_effet, 'on_crit_' + num_effet, 'on_death_' + num_effet, 'random_' + num_effet, 'quantity_' + num_effet, 'all_' + num_effet,
            'self_hp_' + num_effet,'target_hp_' + num_effet, 'damage_' + num_effet, 'note_' + num_effet]
        
    for col in col_list:
        df_spells[col].fillna(-1, inplace=True)

C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1057030776.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_spells.fillna(0, inplace=True)


In [10]:
    # on refait pareil pour l'effect
df_spells['effect_1'] = df_spells['effect_1'].apply(changement_effect, number="_1")
df_spells['effect_2'] = df_spells['effect_2'].apply(changement_effect, number="_2")
df_spells['effect_3'] = df_spells['effect_3'].apply(changement_effect, number="_3")
df_spells['effect_4'] = df_spells['effect_4'].apply(changement_effect, number="_4")
df_spells['effect_5'] = df_spells['effect_5'].apply(changement_effect, number="_5")

In [11]:
for effect in ['effect_1', 'effect_2', 'effect_3', 'effect_4', 'effect_5']:
    df_spells = extraire_variables_imbriquees(df_spells, effect)
    
df_spells.drop([0], axis=1, inplace=True) 

In [12]:
# si vide, pas d'effet, donc false
for num_effet in range(1,6):
    num_effet = str(num_effet)
    col_list = ['id_' + num_effet, 'url_' + num_effet, 'name_' + num_effet,
            'is_buff_' + num_effet, 'type_' + num_effet]
    
    col_delete = ['description_' + num_effet, 'icon_filename_' + num_effet]
    
    df_spells.drop(col_delete, axis=1, inplace=True)
        
    for col in col_list:
        df_spells[col].fillna("-1", inplace=True)
        

# Cleanage
 
df_spells.drop(['count', 'com2us_id' , 'max_level'], axis=1, inplace=True)



C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1522398395.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_spells[col].fillna("-1", inplace=True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1522398395.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_spells[col].fillna("-1", inplace=True)
C:\Users\Kevin\AppData\Local\Temp\ipykernel_2308\1522398395.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '-1' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_spells[col].fillna("-1", inplace=True)
C:\Users\Kevin\AppData\Local\Temp\

In [13]:
df_spells = extraire_variables_imbriquees(df_spells, 'upgrades')

df_spells.rename(columns={0 : 'upgrade_1', 1 : 'upgrade_2', 2: 'upgrade_3', 3: 'upgrade_4', 4: 'upgrade_5', 5 : 'upgrade_6', 6: 'upgrade_7', 7: 'upgrade_8'}, inplace=True)

def changement_upgrade(x):
    if x != 0 and x!= -1 and x != "-1":
        effect = x['effect']
        amount = x['amount']
    else:
        effect = -1
        amount = -1
    return effect, amount


for upgrade in ['upgrade_1', 'upgrade_2', 'upgrade_3', 'upgrade_4', 'upgrade_5', 'upgrade_6', 'upgrade_7', 'upgrade_8']:
    df_spells[upgrade].fillna(-1, inplace=True)
    df_spells[[upgrade + '_effect', upgrade + '_amount']] = df_spells[upgrade].apply(changement_upgrade).apply(pd.Series)
df_spells

,name,description,slot,cooltime,hits,passive,aoe,multiplier_formula,multiplier_formula_raw,scales_with,...,upgrade_4_effect,upgrade_4_amount,upgrade_5_effect,upgrade_5_amount,upgrade_6_effect,upgrade_6_amount,upgrade_7_effect,upgrade_7_amount,upgrade_8_effect,upgrade_8_amount
id,,,,,,,,,,,,,,,,,,,,,
1,Forbidden Ingredient (Passive),Increases the Skill Level of the monster by 1 ...,1,0.0,0,True,False,,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,Special Ingredient (Passive),"The XP granted when used as Power-up material,...",1,0.0,0,True,False,,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,Best Ingredient (Passive),The XP granted will additionally increase by 5...,2,0.0,0,True,False,,[],[],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",[ATK],...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
5,Splinter Attack,Hurls sharp pieces of lumber and stuns the ene...,1,0.0,1,False,False,3.6*{ATK},"[[""ATK"", ""*"", 3.6]]",[ATK],...,Damage +{0}%,10,Effect Rate +{0}%,10,Damage +{0}%,10,Effect Rate +{0}%,15,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3849,Pulverize,Strikes the enemy with a hammer and prevents t...,1,0.0,1,False,False,2.4*{ATK} + 2.8*{DEF},"[[""ATK"", ""*"", 2.4], [""+""], [""DEF"", ""*"", 2.8]]","[ATK, DEF]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3850,Spinning Smash,"Swings the hammer, attacking all enemies to in...",2,3.0,1,False,True,3.7*{DEF},"[[""DEF"", ""*"", 3.7]]",[DEF],...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3851,Powerful Smash,"Slams the enemy with a hammer, decreasing the ...",2,3.0,1,False,False,3.7*{ATK} + 5.0*{DEF},"[[""ATK"", ""*"", 3.7], [""+""], [""DEF"", ""*"", 5.0]]","[ATK, DEF]",...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [14]:
df_spells.to_excel('swarfarm_spells.xlsx', index=False)
df_spells.to_csv('swarfarm_spells.csv', index=False)

In [15]:
df_spells['name_1'].unique()

array(['-1', 'Stun', 'Glancing Hit', 'Decrease ATB', 'Decrease ATK',
       'Decrease DEF', 'Continuous DMG', 'Sleep', 'Decrease SPD',
       'Increase ATK', 'Shield', 'Decrease Damage', 'Reflect DMG', 'Heal',
       'Freeze', 'Debuff Bonus Damage', 'Cleanse', 'Increase ATB',
       'Revive', 'Disturb HP Recovery', 'Self-Heal', 'Increase SPD',
       'Ally Attack', 'Transfer Debuff',
       'Increase Attack Critical Chance', 'Ignore DEF',
       'Detonate Continuous Damage', 'Immunity', 'Remove Buff', 'Provoke',
       'Increase DEF', 'Reduce Cooltime', 'Additional Turn', 'Self-Harm',
       'Counter', 'Increase Damage', 'Silence', 'Ignore Damage Reduction',
       'Additional Attack', 'Increase CRI Rate', 'Passive Reflect DMG',
       'Guaranteed Critical Hit', 'Guaranteed Crushing Hit',
       'Beneficial Effects Blocked', 'Destroy HP', 'Absorb ATB',
       'Accumulate ATK', 'Increase Cooltime', 'Berserk', 'Brand',
       'Steal Buff', 'Elemental Advantage', 'Endure', 'Redistribute H

In [16]:
def verif_effet(df, effet):
    df['verif1'] = df['name_1'].str.contains(effet).astype('int')
    df['verif2'] = df['name_2'].str.contains(effet).astype('int')
    df['verif3'] = df['name_3'].str.contains(effet).astype('int')
    df['verif4'] = df['name_4'].str.contains(effet).astype('int')
    df['verif5'] = df['name_5'].str.contains(effet).astype('int')
    df['verif_g'] = df['verif1'] + df['verif2'] + df['verif3'] + df['verif4'] + df['verif5']
    
    def return_bool(x):
        if x >= 1:
            x = True
        else:
            x = False
        return x
    
    df['verif_g'] = df['verif_g'].apply(return_bool)
    
    df.drop(['verif1', 'verif2', 'verif3', 'verif4', 'verif5'], axis=1, inplace=True)
    
    return df

In [17]:
df_spells2 = verif_effet(df_spells, 'Threat')

In [18]:
df_filtre = df_spells2[df_spells2['verif_g'] == True]

In [19]:
list_mob = []
for index, element in df_filtre['used_on'].items():
    for id_mob in element:
        if id_mob not in list_mob:
            list_mob.append(id_mob)

In [20]:
# df_mob[df_mob['id'].isin(list_mob)]

In [21]:
from sqlalchemy import create_engine, text, Float, BigInteger
from os import environ

# https://betterprogramming.pub/how-to-execute-plain-sql-queries-with-sqlalchemy-627a3741fdb1

DB = environ.get('API_SQL')

def init_connection():
    engine = create_engine(DB, echo=False)
    return engine.connect()

conn = init_connection()

In [22]:
df_mob.to_sql('sw_ref_monsters', conn, if_exists='replace')
# df_spells.to_sql('sw_ref_spells', conn, if_exists='replace')

320

In [33]:
df_spells.drop(['upgrade_1', 'upgrade_2', 'upgrade_3', 'upgrade_4', 'upgrade_5', 'upgrade_6', 'upgrade_7', 'upgrade_8'], axis=1).to_sql('sw_ref_spells',
                                                                                                                                        conn,
                                                                                                                                        if_exists='replace')

20